In [7]:
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [8]:
cd gdrive/My\ Drive

/content/gdrive/My Drive


In [11]:
cd Final\ Demo/iCAN/tools

/content/gdrive/My Drive/Final Demo/iCAN/tools


In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

%matplotlib inline
import _init_paths
from PIL import Image
import matplotlib.pyplot as plt
from ult.config import cfg

import _init_paths
import pickle
import json
import numpy as np
import cv2
import os
import sys

CLASSES = ('__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus','train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter','bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack','umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite','baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl','banana', 'apple', 'sandwich', 'orange', 'broccoli','carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table','toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven','toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier','toothbrush')

In [14]:
Detection = pickle.load( open( cfg.ROOT_DIR + "/demo/HOI_Detection.pkl", "rb" ) )

UnicodeDecodeError: ignored

In [4]:
outputdir=cfg.ROOT_DIR+"/Outputs/"
for image_id in os.listdir(cfg.ROOT_DIR + '/demo/'):
  print(image_id)
  if(image_id=='Object_Detection.pkl' or image_id=='HOI_Detection.pkl'):
    continue

  cc = plt.get_cmap('hsv', lut=6)
  dpi = 80

  im_file = cfg.ROOT_DIR + '/demo/' + image_id
  im_data = plt.imread(im_file)
  height, width, nbands = im_data.shape
  figsize = width / float(dpi), height / float(dpi)
  fig = plt.figure(figsize=figsize)
  ax = fig.add_axes([0, 0, 1, 1])
  ax.axis('off')
  ax.imshow(im_data, interpolation='nearest')

  HO_dic = {}
  HO_set = set()
  count = 0


  for ele in Detection:
      if (ele['image_id'] == image_id):
          action_count = -1
          H_box = ele['person_box'] 
          
          if tuple(H_box) not in HO_set:
              HO_dic[tuple(H_box)] = count
              HO_set.add(tuple(H_box))
              count += 1 
          
          show_H_flag = 0
          
          if ele['smile'][4] > 0.5:
              ax.text(H_box[0] + 10, H_box[1] + 25 + action_count * 35,
              'smile, ' + "%.2f" % ele['smile'][4] ,
              bbox=dict(facecolor=cc(HO_dic[tuple(H_box)])[:3], alpha=0.5),
              fontsize=16, color='white')
              action_count += 1 
              show_H_flag = 1

          if ele['stand'][4] > 0.5:
              ax.text(H_box[0] + 10, H_box[1] + 25 + action_count * 35,
              'stand, ' + "%.2f" % ele['stand'][4] ,
              bbox=dict(facecolor=cc(HO_dic[tuple(H_box)])[:3], alpha=0.5),
              fontsize=16, color='white')
              action_count += 1             
              show_H_flag = 1

          if ele['run'][4] > 0.5:
              ax.text(H_box[0] + 10, H_box[1] + 25 + action_count * 35,
              'run, ' + "%.2f" % ele['run'][4] ,
              bbox=dict(facecolor=cc(HO_dic[tuple(H_box)])[:3], alpha=0.5),
              fontsize=16, color='white')
              action_count += 1  
              show_H_flag = 1

          if ele['walk'][4] > 0.5:
              ax.text(H_box[0] + 10, H_box[1] + 25 + action_count * 35,
              'walk, ' + "%.2f" % ele['walk'][4] ,
              bbox=dict(facecolor=cc(HO_dic[tuple(H_box)])[:3], alpha=0.5),
              fontsize=16, color='white')
              action_count += 1  
              show_H_flag = 1
          
          for action_key, action_value in ele.iteritems():
              if (action_key.split('_')[-1] != 'agent') and action_key != 'image_id' and action_key != 'person_box':
                  if (not np.isnan(action_value[0])) and (action_value[5] > 0.05):
                      O_box = action_value[:4]
                      
                      action_count += 1
                      
                      if tuple(O_box) not in HO_set:
                          HO_dic[tuple(O_box)] = count
                          HO_set.add(tuple(O_box))
                          count += 1      
                  
                      ax.add_patch(
                      plt.Rectangle((H_box[0], H_box[1]),
                                    H_box[2] - H_box[0],
                                    H_box[3] - H_box[1], fill=False,
                                    edgecolor=cc(HO_dic[tuple(H_box)])[:3], linewidth=3)
                      )
                      text = action_key.split('_')[0] + ' ' + CLASSES[np.int(action_value[4])] + ', ' + "%.2f" % action_value[5]

                      ax.text(H_box[0] + 10, H_box[1] + 25 + action_count * 35,
                          text,
                          bbox=dict(facecolor=cc(HO_dic[tuple(O_box)])[:3], alpha=0.5),
                          fontsize=16, color='white')

                      ax.add_patch(
                      plt.Rectangle((O_box[0], O_box[1]),
                                    O_box[2] - O_box[0],
                                    O_box[3] - O_box[1], fill=False,
                                    edgecolor=cc(HO_dic[tuple(O_box)])[:3], linewidth=3)
                      )
                      ax.set(xlim=[0, width], ylim=[height, 0], aspect=1)
          if show_H_flag == 1:
              ax.add_patch(
              plt.Rectangle((H_box[0], H_box[1]),
                            H_box[2] - H_box[0],
                            H_box[3] - H_box[1], fill=False,
                            edgecolor=cc(HO_dic[tuple(H_box)])[:3], linewidth=3)
              )
  plt.savefig(outputdir+image_id)


NameError: ignored

In [0]:
import cv2
import numpy as np
import glob

In [0]:

def createvideo():
  img_array = []
  img1 = cv2.imread("../outputs/stand/0.png")
  height1, width1, layers1 = img1.shape
  size = (width1,height1)
  for filename in glob.glob('../outputs/stand/*.png'):
      img = cv2.imread(filename)
      img_array.append(img)
  
  
  out = cv2.VideoWriter('../outputs/Videos/stand.avi',cv2.VideoWriter_fourcc(*'DIVX'), 5, size)
  
  for i in range(len(img_array)):
      out.write(img_array[i])
  out.release()
  print("Done")